# Laser Basic Tests

## Objective

The objective of this notebooks is to analyze the effectiveness of the JRT BB2X Laser Distance Sensor by Chengdu JRT Meter Technology Co. Ltd, at measuring distances of different lengths while stationary, under sunlight and dark conditions.

## Overview

- Like the TOF sensor's basic tests, I placed the sensor at fixed intervals of 0.5m, from 0.5m to 5.0m facing a wall, and allowed it to generate roughly 500 points of distance measurements.
- Both tests were conducted outdoors, but one in the very early morning with no sunlight, and when at mid day when the sun was its brightest.
- The data was collected through the default software provided and extracted into text files.

### Documentation

### Code

- As before, much of the code has been separated into different modules:
- [`data_loader`](./data_loader.py) contains helper functions to extract the data from the necessary files.
- [`graphing`](./graphing.py) contains helper functions to plot the graphs below.
- [`preprocessing`](./preprocessing.py) contains utility functions that manipulate the data.

In [1]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd

from data_loader import get_laser_data_from_txt, get_data_from_folder
from graphing import BasicGraphs
from laser import Laser
from preprocessing import DataCleaner, get_mean_measurements, get_standard_deviations

In [2]:
graphs = BasicGraphs()
data_cleaner = DataCleaner()
laser_utils = Laser()

In [3]:
ACTUAL_DISTANCES = [0.5 * i for i in range(1, 11)]
INDOORS_DATA_PATH = pathlib.Path("./data/laser_basic_tests/indoors/")

indoors_raw_data = get_data_from_folder(INDOORS_DATA_PATH, get_laser_data_from_txt)
for i in range(len(indoors_raw_data)):
    for j, protocol in enumerate(indoors_raw_data[i]):
        distance = laser_utils.get_distance(protocol)
        indoors_raw_data[i][j] = distance

print(indoors_raw_data)

ValueError: invalid literal for int() with base 16: ''